In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import MultiLabelBinarizer
from collections import Counter
from itertools import chain
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from mlxtend.frequent_patterns import apriori, association_rules
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
df_ap= pd.read_csv('../data/skills_only_data.csv')

In [3]:
# @title One-time precomputation of co-occurrence

#df: job_title + binary skill columns
skill_names = df_ap.columns.to_list()

# map lowercase name -> actual column name (for user input)
skill_name_map = {s.lower(): s for s in skill_names}

# matrix for co-occurrence fallback
X = df_ap.values.astype(np.uint8)

# co-occurrence counts for all skill pairs
co_counts = X.T @ X              # shape (n_skills, n_skills)

# per-skill counts and supports
skill_counts = co_counts.diagonal()
n_jobs = len(df_ap)
skill_support = skill_counts / n_jobs

# index for quick lookup
skill_index = {s: i for i, s in enumerate(skill_names)}

print("Precomputed co-occurrence matrix for", len(skill_names), "skills.")


Precomputed co-occurrence matrix for 2347 skills.


In [4]:
n_jobs = len(df_ap)

print("Apriori input shape:", df_ap.shape)

# =====================================================
# 1. EXPLORE DIFFERENT SUPPORT LEVELS (for your report)
# =====================================================
support_values = [0.02, 0.015, 0.01, 0.0075, 0.005]  # 2%, 1.5%, 1%, 0.75%, 0.5%

for ms in support_values:
    fi = apriori(
        df_ap,
        min_support=ms,
        use_colnames=True,
        max_len=2,        # singletons + pairs
        verbose=0
    )
    fi["itemset_len"] = fi["itemsets"].apply(len)

    rules_tmp = association_rules(fi, metric="confidence", min_threshold=0.4)

    print(f"\n=== min_support = {ms:.4f} (~{int(ms*n_jobs)} jobs) ===")
    print("  1-itemsets:", (fi["itemset_len"] == 1).sum())
    print("  2-itemsets:", (fi["itemset_len"] == 2).sum())
    print("  rules (conf >= 0.4):", len(rules_tmp))

# =====================================================
# 2. CHOOSE A FINAL CONFIG (more generous than before)
# =====================================================

# Example: require at least 50 jobs (~1% support)
min_support_count = 50
min_support = min_support_count / n_jobs

print(f"\nUsing FINAL min_support = {min_support:.4f} (~{min_support_count} jobs)")

frequent_itemsets = apriori(
    df_ap,
    min_support=min_support,
    use_colnames=True,
    max_len=2,        # pairs only for now
    verbose=1
)

frequent_itemsets["itemset_len"] = frequent_itemsets["itemsets"].apply(len)

print("Itemsets by length:")
print(frequent_itemsets["itemset_len"].value_counts())
display(
    frequent_itemsets
        .sort_values("support", ascending=False)
        .head(20)
)

pairs = frequent_itemsets[frequent_itemsets["itemset_len"] == 2]
print("Number of frequent pairs:", len(pairs))

# ---------- association rules ----------
rules = association_rules(
    frequent_itemsets,
    metric="confidence",
    min_threshold=0.4   # a bit looser here
)

# add lengths
rules["antecedent_len"] = rules["antecedents"].apply(len)
rules["consequent_len"] = rules["consequents"].apply(len)

# keep A -> B rules (antecedent ≥1 skill, consequent = 1 skill)
rules = rules[
    (rules["antecedent_len"] >= 1) &
    (rules["consequent_len"] == 1)
]

# OPTIONAL: tighten again by confidence / lift
rules = rules[
    (rules["confidence"] >= 0.5) &    # final filter
    (rules["lift"] >= 1.05)
]

rules_sorted = rules.sort_values(["lift", "confidence"], ascending=False)

print("FINAL number of rules:", len(rules_sorted))
display(rules_sorted.head(20))


Apriori input shape: (2112, 2347)


/Users/anshikabajpai/Desktop/data_mining/CareerPathGuidanceTool/.venv/lib/python3.12/site-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(



=== min_support = 0.0200 (~42 jobs) ===
  1-itemsets: 191
  2-itemsets: 470
  rules (conf >= 0.4): 223


/Users/anshikabajpai/Desktop/data_mining/CareerPathGuidanceTool/.venv/lib/python3.12/site-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(



=== min_support = 0.0150 (~31 jobs) ===
  1-itemsets: 254
  2-itemsets: 845
  rules (conf >= 0.4): 339


...


=== min_support = 0.0050 (~10 jobs) ===
  1-itemsets: 651
  2-itemsets: 4834
  rules (conf >= 0.4): 1426

Using FINAL min_support = 0.0237 (~50 jobs)
Processing 25760 combinations | Sampling itemset size 2
Itemsets by length:
itemset_len
2    350
1    161
Name: count, dtype: int64


/Users/anshikabajpai/Desktop/data_mining/CareerPathGuidanceTool/.venv/lib/python3.12/site-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets,itemset_len
31,0.424716,(communication),1
40,0.290246,(data analysis),1
109,0.265152,(problem solving),1
138,0.237689,(sql),1
114,0.235795,(python),1
301,0.198390,"(communication, problem solving)",2
113,0.193182,(project management),1
134,0.178977,(software development),1
22,0.178977,(business unit),1
142,0.178504,(systems engineering),1


Number of frequent pairs: 350
FINAL number of rules: 109


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski,antecedent_len,consequent_len
134,(docker),(kubernetes),0.048769,0.061553,0.033617,0.689320,11.198805,1.0,0.030616,3.020626,0.957396,0.438272,0.668943,0.617737,1,1
135,(kubernetes),(docker),0.061553,0.048769,0.033617,0.546154,11.198805,1.0,0.030616,2.095933,0.970438,0.438272,0.522885,0.617737,1,1
80,(css),(javascript),0.026042,0.090436,0.023674,0.909091,10.052356,1.0,0.021319,10.005208,0.924599,0.255102,0.900052,0.585436,1,1
6,(ai),(machine learning),0.048769,0.100852,0.043087,0.883495,8.760290,1.0,0.038169,7.717685,0.931265,0.404444,0.870427,0.655363,1,1
120,(r),(data science),0.057292,0.069129,0.029356,0.512397,7.412204,1.0,0.025396,1.909075,0.917662,0.302439,0.476186,0.468527,1,1
36,(cisco),(network engineering),0.036458,0.098485,0.026042,0.714286,7.252747,1.0,0.022451,3.155303,0.894742,0.239130,0.683073,0.489354,1,1
83,(vulnerability management),(cybersecurity),0.033144,0.109375,0.026042,0.785714,7.183673,1.0,0.022417,4.156250,0.890304,0.223577,0.759398,0.511905,1,1
174,(r),(tableau),0.057292,0.076705,0.030777,0.537190,7.003367,1.0,0.026382,1.994978,0.909307,0.298165,0.498741,0.469212,1,1
118,(data science),(machine learning),0.069129,0.100852,0.042614,0.616438,6.112290,1.0,0.035642,2.344207,0.898508,0.334572,0.573416,0.519487,1,1
107,(data mining),(data visualization),0.055871,0.118371,0.038826,0.694915,5.870644,1.0,0.032212,2.889783,0.878758,0.286713,0.653953,0.511458,1,1


## Recommendation

Even if association rules don't match completely, recommendation can be given based on co-occurrence

In [5]:
# @title function for rules & correlations
def map_user_skills_to_columns(user_skills):
    """Map user input skills (strings) to actual df_ap columns."""
    mapped = []
    unknown = []
    for s in user_skills:
        key = s.strip().lower()
        if key in skill_name_map:
            mapped.append(skill_name_map[key])
        else:
            unknown.append(s)
    return mapped, unknown


def recommend_skills_for_user(
    user_skills,
    rules_df,
    top_n=10,
    min_confidence=0.4,
    min_lift=1.0,
):
    """
    Always returns some recommended skills, even if association rules don't match.

    Output columns:
    ['suggested_skill', 'score', 'source', 'support', 'confidence', 'lift']
    """

    # --- normalize & map user skills ---
    mapped_user_skills, unknown = map_user_skills_to_columns(user_skills)
    user_set = set(mapped_user_skills)

    print("User skills mapped to columns:", mapped_user_skills)
    if unknown:
        print("Unknown skills (not in data):", unknown)


    # 1) RULE-BASED RECOMMENDATIONS (FORWARD + REVERSE)

    if rules_df is not None and not rules_df.empty and user_set:
        rules_filt = rules_df.copy()

        # basic thresholds
        rules_filt = rules_filt[
            (rules_filt["confidence"] >= min_confidence)
            & (rules_filt["lift"] >= min_lift)
        ].copy()

        def has_overlap(skill_set):
            return len(set(skill_set) & user_set) > 0

        # forward: user has antecedent(s), suggest consequent
        mask_fwd = rules_filt["antecedents"].apply(has_overlap)
        # reverse: user has consequent(s), suggest antecedent
        mask_rev = rules_filt["consequents"].apply(has_overlap)

        rules_filt["direction"] = np.where(mask_fwd, "forward",
                                   np.where(mask_rev, "reverse", "none"))
        rules_filt = rules_filt[rules_filt["direction"] != "none"]

        if not rules_filt.empty:
            # choose what to suggest based on direction
            def get_suggested(row):
                if row["direction"] == "forward":
                    return list(row["consequents"])[0]
                else:
                    return list(row["antecedents"])[0]

            rules_filt["suggested_skill"] = rules_filt.apply(get_suggested, axis=1)

            # drop skills user already has
            rules_filt = rules_filt[~rules_filt["suggested_skill"].isin(user_set)]

            # score = confidence * lift
            rules_filt["score"] = rules_filt["confidence"] * rules_filt["lift"]
            rules_filt["source"] = "rules"

            rules_recs = (
                rules_filt
                .sort_values(["score", "support"], ascending=False)
                .drop_duplicates(subset=["suggested_skill"])
                [["suggested_skill", "score", "source", "support", "confidence", "lift"]]
                .head(top_n)
            )

            if not rules_recs.empty:
                return rules_recs


    # 2) CO-OCCURRENCE FALLBACK (P(candidate | user_skill))

    if user_set:
        candidate_skills = [s for s in skill_names if s not in user_set]
        cand_scores = {}

        for cand in candidate_skills:
            j = skill_index[cand]
            scores = []
            for u in user_set:
                i = skill_index[u]
                if skill_counts[i] > 0:
                    # P(cand | u) = count(u & cand) / count(u)
                    scores.append(co_counts[i, j] / skill_counts[i])
            if scores:
                cand_scores[cand] = float(np.mean(scores))

        if cand_scores:
            cooc_recs = (
                pd.DataFrame(
                    [(skill, score) for skill, score in cand_scores.items()],
                    columns=["suggested_skill", "score"],
                )
                .sort_values("score", ascending=False)
                .head(top_n)
            )

            cooc_recs["source"] = "cooccurrence"
            cooc_recs["support"] = cooc_recs["suggested_skill"].map(
                dict(zip(skill_names, skill_support))
            )
            cooc_recs["confidence"] = np.nan
            cooc_recs["lift"] = np.nan

            return cooc_recs


    # 3) GLOBAL FREQUENCY FALLBACK (if user skills don't map)

    global_recs = (
        pd.DataFrame(
            {"suggested_skill": skill_names, "score": skill_support}
        )
        .sort_values("score", ascending=False)
        .head(top_n)
    )

    global_recs["source"] = "global_frequency"
    global_recs["support"] = global_recs["score"]
    global_recs["confidence"] = np.nan
    global_recs["lift"] = np.nan

    return global_recs


In [6]:
# @title test cases

test_cases = [
    ("Unknown",
      ["tableau"]),

    ("Unknown",
     ["python", "sql", "data analysis"]),

    ("Classic data-analysis stack",
     ["python", "sql", "data analysis"]),

    ("Junior data analyst (Excel-heavy)",
     ["excel", "sql", "data visualization"]),

    ("BI / dashboard person",
     ["tableau", "power bi", "data visualization"]),

    ("Web dev – front end",
     ["html", "css", "javascript"]),

    ("Web dev – full stack JS",
     ["javascript", "node", "react", "sql"]),

    ("DevOps / cloud",
     ["linux", "docker", "kubernetes", "aws"]),

    ("Backend Java engineer",
     ["java", "spring", "sql"]),

    ("Cybersecurity / infosec",
     ["cybersecurity", "network security", "linux"]),

    ("QA / testing",
     ["software testing", "selenium", "automation"]),

    ("Totally random + one real skill",
     ["knitting", "underwater basket weaving", "python"]),
]


In [7]:
for label, user_skills in test_cases:
    print("=" * 80)
    print(label)
    print("User skills:", user_skills)

    recs = recommend_skills_for_user(
        user_skills=user_skills,
        rules_df=rules_sorted,
        top_n=8,
        min_confidence=0.5,
        min_lift=1.05,
    )

    display(recs)


Unknown
User skills: ['tableau']
User skills mapped to columns: ['tableau']


,suggested_skill,score,source,support,confidence,lift
174,r,3.762139,rules,0.030777,0.537190,7.003367
128,data visualization,3.481694,rules,0.049242,0.641975,5.423407
177,sql,2.545079,rules,0.059659,0.777778,3.272244
103,data analysis,2.322238,rules,0.062973,0.820988,2.828590
32,business unit,1.801971,rules,0.043561,0.567901,3.173035
171,python,1.397656,rules,0.044034,0.574074,2.434627


Unknown
User skills: ['python', 'sql', 'data analysis']
User skills mapped to columns: ['python', 'sql', 'data analysis']


,suggested_skill,score,source,support,confidence,lift
166,r,3.764467,rules,0.053977,0.942149,3.995619
167,spark,3.045794,rules,0.023674,0.847458,3.594037
177,tableau,2.545079,rules,0.059659,0.777778,3.272244
89,data visualization,2.454247,rules,0.099905,0.844000,2.907876
112,data mining,2.447435,rules,0.042614,0.762712,3.208859
119,data science,2.363806,rules,0.051610,0.746575,3.166199
130,data warehouse,2.347564,rules,0.029356,0.746988,3.142706
96,power bi,2.084225,rules,0.059659,0.777778,2.679717


Classic data-analysis stack
User skills: ['python', 'sql', 'data analysis']
User skills mapped to columns: ['python', 'sql', 'data analysis']


...

,suggested_skill,score,source,support,confidence,lift
83,vulnerability management,5.644315,rules,0.026042,0.785714,7.183673
147,windows,2.946364,rules,0.036458,0.616000,4.783059
156,network engineering,2.910127,rules,0.025095,0.535354,5.435897


QA / testing
User skills: ['software testing', 'selenium', 'automation']
User skills mapped to columns: ['software testing', 'selenium', 'automation']


,suggested_skill,score,source,support,confidence,lift
1584,python,0.359107,cooccurrence,0.114583,NaN,NaN
1910,software development,0.314650,cooccurrence,0.057765,NaN,NaN
289,communication,0.304534,cooccurrence,0.061080,NaN,NaN
1032,java,0.251938,cooccurrence,0.116004,NaN,NaN
1967,sql,0.237883,cooccurrence,0.116477,NaN,NaN
311,computer science,0.226389,cooccurrence,0.091856,NaN,NaN
628,engineering,0.225542,cooccurrence,0.099432,NaN,NaN
2134,testing,0.217468,cooccurrence,0.106061,NaN,NaN


Totally random + one real skill
User skills: ['knitting', 'underwater basket weaving', 'python']
User skills mapped to columns: ['python']
Unknown skills (not in data): ['knitting', 'underwater basket weaving']


,suggested_skill,score,source,support,confidence,lift
166,r,3.764467,rules,0.053977,0.942149,3.995619
167,spark,3.045794,rules,0.023674,0.847458,3.594037
119,data science,2.363806,rules,0.051610,0.746575,3.166199
170,statistics,1.958782,rules,0.033144,0.679612,2.882208
148,machine learning,1.938342,rules,0.068182,0.676056,2.867130
8,ai,1.848451,rules,0.032197,0.660194,2.799860
109,data mining,1.713259,rules,0.035511,0.635593,2.695528
104,data engineering,1.515070,rules,0.024621,0.597701,2.534829


In [8]:
rules_sorted.to_csv("../data/association_rules.csv", index=False)

# Job role recommendation

In [9]:
df1= pd.read_csv('../data/skills_with_jobtitle_data.csv')

In [10]:
# @title job role recommendation based on user skills
# cached once
skill_cols = df1.columns.drop("job_title")
skill_name_map_for_roles = {s.lower(): s for s in skill_cols}
skill_index_for_roles = {s: i for i, s in enumerate(skill_cols)}
X_jobs = df1[skill_cols].values.astype(np.uint8)   # job-skill matrix


def suggest_job_roles(user_skills,
                      df_jobs=df1,
                      top_n=15,
                      min_overlap=1):
    """
    Given a list of user skills (strings), return job titles that best match.
    Uses simple set overlap + Jaccard similarity on the binary skill matrix.
    """
    # --- map user skills to existing columns ---
    mapped = []
    unknown = []
    for s in user_skills:
        key = s.strip().lower()
        if key in skill_name_map_for_roles:
            mapped.append(skill_name_map_for_roles[key])
        else:
            unknown.append(s)

    print("User skills mapped to columns:", mapped)
    if unknown:
        print("Unknown (not in data):", unknown)

    if not mapped:
        print("No user skills matched any columns.")
        return pd.DataFrame()

    # user skill vector
    user_vec = np.zeros(len(skill_cols), dtype=np.uint8)
    for sk in mapped:
        user_vec[skill_index_for_roles[sk]] = 1

    # --- overlap & similarity ---
    # overlap = number of shared skills with each job
    overlap = X_jobs @ user_vec

    # counts
    job_skill_counts = X_jobs.sum(axis=1)
    user_skill_count = user_vec.sum()

    # avoid division by zero
    union = job_skill_counts + user_skill_count - overlap
    union[union == 0] = 1

    # Jaccard similarity
    jaccard = overlap / union

    # coverage metrics
    coverage_user = overlap / user_skill_count          # how much of user set covered
    coverage_job = np.divide(
        overlap,
        job_skill_counts,
        out=np.zeros_like(overlap, dtype=float),
        where=job_skill_counts != 0,
    )

    # --- build results per job posting ---
    results = df_jobs[["job_title"]].copy()
    results["overlap"] = overlap
    results["jaccard"] = jaccard
    results["coverage_user"] = coverage_user
    results["coverage_job"] = coverage_job

    # keep only jobs with at least some overlap
    results = results[results["overlap"] >= min_overlap]

    if results.empty:
        print("No jobs share any skills with this skill set.")
        return results

    # --- aggregate by job_title (so repeated postings collapse) ---
    agg = (
        results.groupby("job_title")
        .agg(
            avg_overlap=("overlap", "mean"),
            max_overlap=("overlap", "max"),
            avg_jaccard=("jaccard", "mean"),
            max_jaccard=("jaccard", "max"),
            avg_coverage_user=("coverage_user", "mean"),
            avg_coverage_job=("coverage_job", "mean"),
            n_postings=("overlap", "count"),
        )
        .reset_index()
    )

    # rank by max_overlap then avg_jaccard
    agg = agg.sort_values(
        ["max_overlap", "avg_jaccard"],
        ascending=False
    ).head(top_n)

    return agg

In [11]:
user_skills = ["python", "sql", "data analysis"]

top_roles = suggest_job_roles(user_skills, top_n=15, min_overlap=2)
display(top_roles)


User skills mapped to columns: ['python', 'sql', 'data analysis']


,job_title,avg_overlap,max_overlap,avg_jaccard,max_jaccard,avg_coverage_user,avg_coverage_job,n_postings
15,BI Business Analyst,3.0,3,0.375000,0.375000,1.000000,0.375000,1
5,Accountant - Data Analyst,3.0,3,0.250000,0.250000,1.000000,0.250000,1
202,Technical Data Analyst,2.5,3,0.234848,0.333333,0.833333,0.268182,2
18,"BUSINESS DATA ANALYST- Austin , TX",3.0,3,0.214286,0.214286,1.000000,0.214286,1
132,Junior Data Analyst,2.5,3,0.204861,0.222222,0.833333,0.218750,2
17,"BI Data Analyst (ThoughtSpot, Tableau, SQL) - ...",3.0,3,0.200000,0.200000,1.000000,0.200000,1
72,Data Analyst with Healthcare- REMOTE,3.0,3,0.200000,0.200000,1.000000,0.200000,1
97,Data Scientist (USA),3.0,3,0.200000,0.200000,1.000000,0.200000,1
77,"Data Analyst-SQL, Python, Visualization tool",3.0,3,0.187500,0.187500,1.000000,0.187500,1
98,Data Scientist - Advana,3.0,3,0.187500,0.187500,1.000000,0.187500,1


In [12]:
user_skills = ["cybersecurity"]

top_roles = suggest_job_roles(user_skills, top_n=15, min_overlap=1)
display(top_roles)

User skills mapped to columns: ['cybersecurity']


,job_title,avg_overlap,max_overlap,avg_jaccard,max_jaccard,avg_coverage_user,avg_coverage_job,n_postings
151,VPN Network Systems Engineer (Secret Clearance...,1.0,1,0.200000,0.200000,1.0,0.200000,1
61,IT Security Analyst,1.0,1,0.173611,0.333333,1.0,0.173611,3
37,Cyber Systems Security Analyst,1.0,1,0.142857,0.142857,1.0,0.142857,1
50,Fire and Security Engineer (Static),1.0,1,0.142857,0.142857,1.0,0.142857,1
78,"Information security analysts - Malver, PA - L...",1.0,1,0.111111,0.111111,1.0,0.111111,1
108,Secret Cleared Network Engineer,1.0,1,0.111111,0.111111,1.0,0.111111,1
15,Business Analyst- NEED LOCAL CANDIDATES,1.0,1,0.100000,0.100000,1.0,0.100000,1
148,Technical Writer/Business Analyst (Onsite),1.0,1,0.100000,0.100000,1.0,0.100000,1
5,Application Security Analyst,1.0,1,0.090909,0.090909,1.0,0.090909,1
39,Cybersecurity Analyst (RMF),1.0,1,0.090909,0.090909,1.0,0.090909,1
